# SageMaker Edge Manager Example

1. [Introduction](#Introduction)
2. [Demo Setup](#Demo-Setup)
3. [Compile Model using SageMaker Neo](#Compile-Model-using-SageMaker-Neo)
    1. [Load pretrained model](#Load-pretrained-model)
6. [Deploy Model using Sagemaker Edge Manager](#Deploy-Model-using-Sagemaker-Edge-Manager)
    1. [Package Model](#Package-Model)
    2. [Create AWS IoT thing](#Create-AWS-IoT-thing)
    3. [Create Device Fleet](#Create-Device-Fleet)
    4. [Create and register client certificate with AWS IoT](#Create-and-register-client-certificate-with-AWS-IoT)
7. [Inference on Edge](#Inference-on-Edge)

**Please note**: There are pricing implications to the use of this notebook. Please refer to [Edge Manager](https://aws.amazon.com/sagemaker/edge-manager/pricing) for more information.

## Demo Setup

We need an AWS account role with SageMaker access. This role is used to give SageMaker access to S3, launch an EC2 instance and send command with Systems Manager.

In [ ]:
#update boto3 version
!pip3 install boto3==1.16.45 
!pip3 install awscli==1.18.209

In [ ]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import botocore
import json

role = get_execution_role()
sess = sagemaker.Session()
region = boto3.Session().region_name

In [ ]:
print(role)

Locate the above printed sagemaker role from IAM console, find and attach the following policies to role:

AmazonEC2FullAccess
AmazonEC2RoleforSSM
AmazonSSMManagedInstanceCore
AmazonSSMFullAccess
AWSIoTFullAccess
You can find more information about how to attach policies to role here - https://docs.aws.amazon.com/IAM/latest/UserGuide/access_policies_manage-attach-detach.html#add-policies-console.

If you try this example with a real device, only attach AWSIoTFullAccess to create certificates on AWS IoT.

We then need an S3 bucket that would be used for storing the model artifacts generated after compilation and packaged artifacts generated after edge packaging job.

We then need an S3 bucket that would be used for storing the model artifacts generated after compilation and packaged artifacts generated after edge packaging job.

In [ ]:
# S3 bucket and folders for saving model artifacts.
# Feel free to specify different bucket/folders here if you wish.
bucket = sess.default_bucket() 
folder = 'DEMO-Sagemaker-Edge'
compilation_output_sub_folder = folder + '/compilation-output'
iot_folder = folder + '/iot'

# S3 Location to save the model artifact after compilation
s3_compilation_output_location = 's3://{}/{}'.format(bucket, compilation_output_sub_folder)

Finally we upload the test image to S3 bucket. This image will be used in inference later.

In [ ]:
#Upload an image for testing purposes. 
darknet_img_path = sess.upload_data('<<IMAGE_PATH>>', bucket, iot_folder)
keras_img_path = sess.upload_data('<<IMAGE_PATH>>', bucket, iot_folder)

## Compile Model using SageMaker Neo

Create Sagemaker client.

In [ ]:
sagemaker_client = boto3.client('sagemaker', region_name=region)

### Download pretrained darknet model

In [ ]:
!wget -O yolov3-tiny.cfg https://github.com/pjreddie/darknet/blob/master/cfg/yolov3-tiny.cfg?raw=true
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

In [ ]:
import tarfile

with tarfile.open('yolov3-tiny.tar.gz', mode='w:gz') as archive:
    archive.add('yolov3-tiny.cfg')
    archive.add('yolov3-tiny.weights')

In [ ]:
darknet_model_path = sess.upload_data('yolov3-tiny.tar.gz', bucket, folder)

**Note**: When calling ``create_compilation_job()`` user is expected to provide all the correct input shapes required by the model for successful compilation. If we are using a different model, we need to specify the framework and data shape correctly..

In [ ]:
darknet_model_data_shape = '{"data":[1,3,416,416]}'
darknet_model_framework = 'darknet'
target_device = 'jetson_xavier' 

In [ ]:
import time
darknet_compilation_job_name = 'Sagemaker-Edge-'+ str(time.time()).split('.')[0]
print('Compilation job for %s started' % darknet_compilation_job_name)

response = sagemaker_client.create_compilation_job(
        CompilationJobName=darknet_compilation_job_name,
        RoleArn=role,
        InputConfig={
            'S3Uri': darknet_model_path,
            'DataInputConfig': darknet_model_data_shape,
            'Framework': darknet_model_framework.upper()
        },
        OutputConfig={
            'S3OutputLocation': s3_compilation_output_location,
            'TargetDevice': target_device 
        },
        StoppingCondition={
            'MaxRuntimeInSeconds': 900
        }
    )

print(response)

# Poll every 30 sec
while True:
    response = sagemaker_client.describe_compilation_job(CompilationJobName=darknet_compilation_job_name)
    if response['CompilationJobStatus'] == 'COMPLETED':
        break
    elif response['CompilationJobStatus'] == 'FAILED':
        raise RuntimeError('Compilation failed')
    print('Compiling ...')
    time.sleep(30)
print('Done!')

## Package Model using Sagemaker Edge Manager

In this section, we will walk through packaging two models that achieve different goals. One is an Image Classification model (from Keras framework) and another is an Object Detection Model from DarkNet framework. This showcases the versatility of SageMaker Edge Manager.

### Package Darknet Model

Before we can deploy the compiled model to edge devices, we need to package the model with Sagemaker Edge Manager cloud service.

In [ ]:
darknet_packaged_model_name = "darknet-model"
darknet_model_version = "1.0"
darknet_model_package = '{}-{}.tar.gz'.format(darknet_packaged_model_name, darknet_model_version)

In [ ]:
darknet_packaging_job_name=darknet_compilation_job_name+"-packaging"
response = sagemaker_client.create_edge_packaging_job(
    RoleArn=role,
    OutputConfig={
        'S3OutputLocation': s3_compilation_output_location,
    },
    ModelName=darknet_packaged_model_name,
    ModelVersion=darknet_model_version,
    EdgePackagingJobName=darknet_packaging_job_name,
    CompilationJobName=darknet_compilation_job_name,
)

print(response)

# Poll every 30 sec
while True:
    job_status = sagemaker_client.describe_edge_packaging_job(EdgePackagingJobName=darknet_packaging_job_name)
    if job_status['EdgePackagingJobStatus'] == 'COMPLETED':
        break
    elif job_status['EdgePackagingJobStatus'] == 'FAILED':
        raise RuntimeError('Edge Packaging failed')
    print('Packaging ...')
    time.sleep(30)
print('Done!')

In [ ]:
darknet_model_data = job_status["ModelArtifact"]

### Download pretrained Keras model

In [ ]:
import tensorflow as tf

model = tf.keras.applications.MobileNetV2()
model.save('mobilenet_v2.h5')

In [ ]:
import tarfile

with tarfile.open('mobilenet_v2.tar.gz', mode='w:gz') as archive:
    archive.add('mobilenet_v2.h5')

In [ ]:
keras_model_path = sess.upload_data('mobilenet_v2.tar.gz', bucket, folder)

**Note**: When calling ``create_compilation_job()`` user is expected to provide all the correct input shapes required by the model for successful compilation. If we are using a different model, we need to specify the framework and data shape correctly..

In [ ]:
keras_model_data_shape = '{"input_1":[1,3,224,224]}'
keras_model_framework = 'keras'
target_device = 'jetson_xavier'

In [ ]:
import time
keras_compilation_job_name = 'Sagemaker-Edge-'+ str(time.time()).split('.')[0]
print('Compilation job for %s started' % keras_compilation_job_name)

response = sagemaker_client.create_compilation_job(
        CompilationJobName=keras_compilation_job_name,
        RoleArn=role,
        InputConfig={
            'S3Uri': keras_model_path,
            'DataInputConfig': keras_model_data_shape,
            'Framework': keras_model_framework.upper()
        },
        OutputConfig={
            'S3OutputLocation': s3_compilation_output_location,
            'TargetDevice': target_device 
        },
        StoppingCondition={
            'MaxRuntimeInSeconds': 900
        }
    )

print(response)

# Poll every 30 sec
while True:
    response = sagemaker_client.describe_compilation_job(CompilationJobName=keras_compilation_job_name)
    if response['CompilationJobStatus'] == 'COMPLETED':
        break
    elif response['CompilationJobStatus'] == 'FAILED':
        raise RuntimeError('Compilation failed')
    print('Compiling ...')
    time.sleep(30)
print('Done!')

### Package Keras Model

In [ ]:
keras_packaged_model_name = "keras-model"
keras_model_version = "1.0"
keras_model_package = '{}-{}.tar.gz'.format(keras_packaged_model_name, keras_model_version)

In [ ]:
keras_packaging_job_name=keras_compilation_job_name+"-packaging"
response = sagemaker_client.create_edge_packaging_job(
    RoleArn=role,
    OutputConfig={
        'S3OutputLocation': s3_compilation_output_location,
    },
    ModelName=keras_packaged_model_name,
    ModelVersion=keras_model_version,
    EdgePackagingJobName=keras_packaging_job_name,
    CompilationJobName=keras_compilation_job_name,
)

print(response)

# Poll every 30 sec
while True:
    job_status = sagemaker_client.describe_edge_packaging_job(EdgePackagingJobName=keras_packaging_job_name)
    if job_status['EdgePackagingJobStatus'] == 'COMPLETED':
        break
    elif job_status['EdgePackagingJobStatus'] == 'FAILED':
        raise RuntimeError('Edge Packaging failed')
    print('Packaging ...')
    time.sleep(30)
print('Done!')

In [ ]:
keras_model_data = job_status["ModelArtifact"]

### Create AWS IoT thing

SageMaker Edge Manager uses AWS IoT core to authenticate the device so we can make calls to SageMaker Edge Manager endpoints in AWS Cloud. 

In order for an Edge device AWS to use AWS services, it is necessary for it to first authenticate. We recommend doing this via AWS IoT based authentication, for more details refer [here](https://docs.aws.amazon.com/iot/latest/developerguide/authorizing-direct-aws.html) and [here](https://aws.amazon.com/blogs/security/how-to-eliminate-the-need-for-hardcoded-aws-credentials-in-devices-by-using-the-aws-iot-credentials-provider/).

In [ ]:
iot_client = boto3.client('iot', region_name=region)

In [ ]:

#You can use an existing IoT Thing and Type, OR even create a new one.
iot_thing_name = '<<IOT_THING_NAME>>'
iot_thing_type = '<<IOT_THING_TYPE>>' #example:'SagemakerEdge'

In [ ]:
iot_client.create_thing_type(
    thingTypeName=iot_thing_type
)

In [ ]:
iot_client.create_thing(
    thingName=iot_thing_name,
    thingTypeName=iot_thing_type
)

### Create Device Fleet

#### Create IAM role for device fleet

Configure an IAM role in your AWS account that will be assumed by the credentials provider on behalf of the devices in your device fleet. 

**Notice**: The Name of the role must start with `SageMaker`.

Go to [IAM console](https://console.aws.amazon.com/iam), create role for IoT, attach the following policies:

- AmazonSageMakerEdgeDeviceFleetPolicy

Add the statement to trust relationship:
```
{
  "Version": "2012-10-17",
  "Statement": [
      {
        "Effect": "Allow",
        "Principal": {"Service": "credentials.iot.amazonaws.com"},
        "Action": "sts:AssumeRole"
      },
      {
        "Effect": "Allow",
        "Principal": {"Service": "sagemaker.amazonaws.com"},
        "Action": "sts:AssumeRole"
      }
  ]
}
```

Note down the role ARN, it will be later used for creating the device fleet.

In [ ]:
device_fleet_name ="<<DEVICE_FLEET_NAME>>" + str(time.time()).split('.')[0]
role_name="AmazonSageMaker-ExecutionRole-20201120T163244"

sagemaker_client.create_device_fleet(
    DeviceFleetName=device_fleet_name,
    RoleArn="<<Full_ARN_of_IAM_Role_created_above>>",
    #Example: "arn:aws:iam::ACCOUNT_ID:role/service-role/AmazonSageMaker-ExecutionRole-2021XXXXXXXXXXXX",
    OutputConfig={
        'S3OutputLocation': s3_compilation_output_location
    }
)

#### Register device to the fleet

In [ ]:
device_name = "sagemaker-edge-demo-device" + str(time.time()).split('.')[0] # device name should be 36 charactors
#example > device_name = "sagemaker-edge-demo-device-012345678"

sagemaker_client.register_devices(
    DeviceFleetName=device_fleet_name,
    Devices=[
        {          
            "DeviceName": device_name,
            "IotThingName": iot_thing_name,
            "Description": "this is a sample virtual device"
        }
    ]
)

### Create and register client certificate with AWS IoT

Create private key, public key, and X.509 certificate files and registers and activates the certificate with AWS IoT. 

In [ ]:
#Create a new cert and register if you dont have an existing one already.
iot_cert = iot_client.create_keys_and_certificate(
    setAsActive=True
)

Save the files and upload to S3 bucket, these files will be used to provide credentials on device to communicate with aws services.

In [ ]:
with open('./iot.pem.crt', 'w') as f:
    for line in iot_cert['certificatePem'].split('\n'):
        f.write(line)
        f.write('\n')

In [ ]:
with open('./iot_key.pem.key', 'w') as f:
    for line in iot_cert['keyPair']['PrivateKey'].split('\n'):
        f.write(line)
        f.write('\n')

In [ ]:
with open('./iot_key_pair.pem.key', 'w') as f:
    for line in iot_cert['keyPair']['PublicKey'].split('\n'):
        f.write(line)
        f.write('\n')

Associate the role alias generated from `create_device_fleet()` with AWS IoT.

In [ ]:
role_alias_name = 'SageMakerEdge-' + device_fleet_name

role_alias = iot_client.describe_role_alias(
    roleAlias=role_alias_name
)

We created and registered a certificate with AWS IoT earlier for successful authentication of your device. Now, we need to create and attach a policy to the certificate to authorize the request for the security token.

In [ ]:
alias_policy = {
  "Version": "2012-10-17",
  "Statement": {
    "Effect": "Allow",
    "Action": "iot:AssumeRoleWithCertificate",
    "Resource": role_alias['roleAliasDescription']['roleAliasArn']
  }
}

In [ ]:
policy_name = 'aliaspolicy-'+ str(time.time()).split('.')[0]
aliaspolicy = iot_client.create_policy(
    policyName=policy_name,
    policyDocument=json.dumps(alias_policy),
)

In [ ]:
iot_client.attach_policy(
    policyName=policy_name,
    target=iot_cert['certificateArn']
    #example > target="arn:aws:iot:us-west-2:ACCOUNTID:cert/dXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
)

Obtain your AWS account-specific endpoint for the credentials provider.

In [ ]:
iot_endpoint = iot_client.describe_endpoint(
    endpointType='iot:CredentialProvider'
)

In [ ]:
endpoint = "https://{}/role-aliases/{}/credentials".format(iot_endpoint['endpointAddress'], role_alias_name)

In [ ]:
print(endpoint)

Get offical Amazon Root CA file and upload to S3 bucket. 

In [ ]:
!wget https://www.amazontrust.com/repository/AmazonRootCA1.pem

Use the endpoint to make an HTTPS request to the credentials provider to return a security token. The following example command uses curl, but you can use any HTTP client.

**Optional: verify the credentials.**


In [ ]:
!curl --cert iot.pem.crt --key iot_key.pem.key --cacert AmazonRootCA1.pem $endpoint

If the certificate can be verified with the endpoint without error, upload certificate files to S3 bucket.

These files will be used in the [Setup Sagemaker Edge Manager Agent](#Setup-Sagemaker-Edge-Manager-Agent) section on EC2/device as Credential Provider.

In [ ]:
root_ca_path = sess.upload_data('AmazonRootCA1.pem', bucket, iot_folder)
device_cert_path = sess.upload_data('iot.pem.crt', bucket, iot_folder)
device_key_path = sess.upload_data('iot_key.pem.key', bucket, iot_folder)

## Inference on Edge

Now you can move on to setting up Greengrass V2 Components to deploy the Neo models, Sagemaker Edge Manager Agent, Sagemaker Edge Manager Client Example(CLI, C++ or Python), and the business/inference logic in a greengrass Lambda functions. 

In [ ]:
#Print the locations of the Artifacts
print(darknet_img_path)
print(darknet_model_data)
print(darknet_model_package)
print(keras_img_path)
print(keras_model_data)
print(keras_model_package)

In [ ]:
#Note the variable names that will be needed below when running the bash scripts
print(role_name, role_alias_name)
print(device_name,device_fleet_name)

Now you have certificates, packaged model, and sagemaker edge config that you can use on the edge device. You can use the bash scripts under the root directory to install Greengrass v2, create custom components in Greengrass v2 and deploy them to the edge device. 

Open a terminal, go to the parent directory of this repo, and run the following:

```
export AWS_PROFILE=<PROFILE-NAME>
export AWS_REGION=<REGION>
export PLATFORM=linux-armv8
export SSH_USER=<USER>
export SSH_HOST=<IP_ADDRESS>
export IOT_THING_NAME=<THING_NAME>
export BUCKET_NAME=<BUCKET_NAME>
export ROLE_NAME=<Use_role_name_from_above_cell>
export ROLE_ALIAS=<Use_role_alias_from_above_cell>
```

Make the scripts executable

```
chmod +x ./scripts/*.sh
```

Install GGv2 on the Edge device using SSH

``` 
./scripts/install-ggv2-ssh-existing-role.sh $AWS_PROFILE $SSH_USER $SSH_HOST $AWS_REGION $IOT_THING_NAME $ROLE_NAME $ROLE_ALIAS 
```

Download Edge Manager Agent Binary onto the device
```
./scripts/download_edge_manager_package.sh $AWS_PROFILE $PLATFORM
```

Add Agent Artifacts to artifacts folder
```
./scripts/add_agent_artifact.sh $AWS_PROFILE $PLATFORM 0.1.0 $AWS_REGION
```

Create a Python Stub of the SageMaker Edge Manager Agent
```
pip install grpcio-tools
pip install --upgrade protobuf
```

Compile and Add the Python artifacts
```
./scripts/compile_add_python_stub_artifacts.sh $PLATFORM aws.sagemaker.edgeManagerPythonClient 0.1.0
```

Update recipes

- In all of the recipe files, replace YOUR_BUCKET_NAME with the value assigned to $BUCKET_NAME

- In components/recipe/aws.sagemaker.edgeManager-0.1.0.yaml, update the endpoint with your region and account number:

```
endpoint: arn:aws:iot:<AWS_REGION>:<ACCOUNT_NUMBER>:rolealias/<DEVICE_FLEET_NAME>
```

- In components/recipe/aws.sagemaker.edgeManager-0.1.0.yaml, update the URI with your region:
```
URI: s3://YOUR_BUCKET_NAME/artifacts/aws.sagemaker.edgeManager/0.1.0/<AWS_REGION>.pem
```

- In components/recipe/aws.sagemaker.edgeManager-0.1.0.yaml, make sure the device fleet name and device name are correct:
```
  DefaultConfiguration:
    deviceName: <DEVICE_NAME>
    deviceFleetName: <DEVICE_FLEET_NAME>
```


Upload your custom components to S3 bucket
```
./scripts/upload_component_version.sh $AWS_PROFILE com.model.darknet 0.1.0 $BUCKET_NAME $AWS_REGION
./scripts/upload_component_version.sh $AWS_PROFILE aws.sagemaker.edgeManager 0.1.0 $BUCKET_NAME $AWS_REGION 
./scripts/upload_component_version.sh $AWS_PROFILE aws.sagemaker.edgeManagerPythonClient 0.1.0 $BUCKET_NAME $AWS_REGION
```

Create a new Greengrass v2 deployment, including the following components:

- com.model.darknet v0.1.0
- aws.sagemaker.edgeManager v0.1.0
- aws.sagemaker.edgeManagerPythonClient v0.1.0 OR any other custom Lambda function that you create. See *greengrass_lambda.py* for example application that takes in Camera stream as input and runs inference using the PythonClient.



    